#### Imports & environnement

In [1]:
# from dotenv import load_dotenv
import pandas as pd
import os

#### Fonctions

In [2]:
# netoyage texte
import re

def nettoyer_caracteres(texte):
     return re.sub(r"[^\wàâäéèêëîïôöùûüÿçæœÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÆŒ' -]", "", str(texte))

In [3]:
def convertir_int(serie, valeur_remplacement=99000):
    def convertir_valeur(valeur):
        try:
            # Tente de convertir la valeur en entier
            return int(valeur)
        except ValueError:
            # En cas d'échec, retourne la valeur de remplacement
            return valeur_remplacement

    # Applique la fonction de conversion à chaque élément de la série
    return serie.apply(convertir_valeur)

#### Read file

In [6]:
# Création du Dataframe
df = pd.read_csv("C:/Users/hayel/Downloads/Analyse_Hadoop/Hadoop/datas/dataw_fro03/dataw_fro03.csv").reset_index(drop=True)

In [7]:
# Vérification de la taille
df.shape

(135277, 25)

In [8]:
# Voir
df.head()

,codcli,genrecli,nomcli,prenomcli,cpcli,villecli,codcde,datcde,timbrecli,timbrecde,...,qte,Colis,libobj,Tailleobj,Poidsobj,points,indispobj,libcondit,prixcond,puobj
0,446,Mme,CHRETIEN,Daniel,14540,BOURGUEBUS,478,2004-10-22 00:00:00,5.0,4.8,...,2.0,1.0,Polo,XL,230,60.0,0,Carton Tete de menagere,0,0
1,446,Mme,CHRETIEN,Daniel,14540,BOURGUEBUS,478,2004-10-22 00:00:00,5.0,4.8,...,2.0,1.0,T-shirt Blanc,L,170,60.0,0,Carton Tete de menagere,0,0
2,446,Mme,CHRETIEN,Daniel,14540,BOURGUEBUS,478,2004-10-22 00:00:00,5.0,4.8,...,1.0,1.0,Montre,Homme,30,150.0,0,Carton Tete de menagere,0,0
3,17860,M.,VERARDO,Anthony,35400,SAINT MALO,21239,2006-10-03 00:00:00,0.0,3.9,...,1.0,1.0,T-shirt Blanc,XL,180,60.0,0,Distingo 500 g,34,0
4,1330,Mme,ROBERT,Yvonne,61000,ALENCON,1386,2004-11-23 00:00:00,4.0,6.5,...,3.0,1.0,Tete de menagere,Confidence,250,100.0,0,Carton Tete de menagere,0,0


#### Nettoyage

In [9]:
# Nan en 0
df = df.fillna(0)

Dates

In [10]:
# turn in date
df['datcde'] = pd.to_datetime(df['datcde'], errors='coerce')

In [11]:
# format
df['datcde'] = df['datcde'].dt.strftime('%Y-%m-%d')

In [12]:
# supprimer les dates inexistantes
df_supprimé = df[(df['datcde'] < '1990-01-01') | (df['datcde'].isna())]

In [13]:
df_supprimé['raison'] = 'Date erronée'

C:\Users\hayel\AppData\Local\Temp\ipykernel_78064\2741674458.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_supprimé['raison'] = 'Date erronée'


In [14]:
# Supprimer les dates inexistantes
df = df = df[(df['datcde'] >= '1990-01-01') & (df['datcde'].notna())]

Code postaux

In [15]:
df['cpcli'] = df['cpcli'].astype(str)
df['DEP'] = df['cpcli'].str[0:2]

In [16]:
df.head()

,codcli,genrecli,nomcli,prenomcli,cpcli,villecli,codcde,datcde,timbrecli,timbrecde,...,Colis,libobj,Tailleobj,Poidsobj,points,indispobj,libcondit,prixcond,puobj,DEP
0,446,Mme,CHRETIEN,Daniel,14540,BOURGUEBUS,478,2004-10-22,5.0,4.8,...,1.0,Polo,XL,230,60.0,0,Carton Tete de menagere,0,0,14
1,446,Mme,CHRETIEN,Daniel,14540,BOURGUEBUS,478,2004-10-22,5.0,4.8,...,1.0,T-shirt Blanc,L,170,60.0,0,Carton Tete de menagere,0,0,14
2,446,Mme,CHRETIEN,Daniel,14540,BOURGUEBUS,478,2004-10-22,5.0,4.8,...,1.0,Montre,Homme,30,150.0,0,Carton Tete de menagere,0,0,14
3,17860,M.,VERARDO,Anthony,35400,SAINT MALO,21239,2006-10-03,0.0,3.9,...,1.0,T-shirt Blanc,XL,180,60.0,0,Distingo 500 g,34,0,35
4,1330,Mme,ROBERT,Yvonne,61000,ALENCON,1386,2004-11-23,4.0,6.5,...,1.0,Tete de menagere,Confidence,250,100.0,0,Carton Tete de menagere,0,0,61


In [17]:
# filtrer
masque_chiffres = df['DEP'].str.isdigit()

In [18]:
df_dep_supprimé = df[~masque_chiffres]

In [19]:
df_dep_supprimé['raison']='DEP érroné'

In [20]:
# ajouter ces lignes au df_supprimé
df_supprimé = pd.concat([df_supprimé, df_dep_supprimé])

In [21]:
df = df[masque_chiffres]

zone de textes

In [22]:
# colonnes à nettoyer :
nettoyer_text = ['nomcli', 'prenomcli', 'villecli']

In [23]:
# enlever les caractères spéciaux
df[nettoyer_text] = df[nettoyer_text].map(nettoyer_caracteres)

In [24]:
df['nomcli']=df['nomcli'].str.title()

In [25]:
df['prenomcli']=df['prenomcli'].str.title()

In [26]:
df['villecli']=df['villecli'].str.title()

Supprimer les .0 dans les index et int

In [27]:
df['codcde'] = df['codcde'].astype(int)

In [28]:
df['codcli'] = df['codcli'].astype(str)

In [29]:
df['codobj'] = df['codobj'].astype(int)

In [30]:
df['qte'] = df['qte'].astype(int)

In [31]:
df.head()

,codcli,genrecli,nomcli,prenomcli,cpcli,villecli,codcde,datcde,timbrecli,timbrecde,...,Colis,libobj,Tailleobj,Poidsobj,points,indispobj,libcondit,prixcond,puobj,DEP
0,446,Mme,Chretien,Daniel,14540,Bourguebus,478,2004-10-22,5.0,4.8,...,1.0,Polo,XL,230,60.0,0,Carton Tete de menagere,0,0,14
1,446,Mme,Chretien,Daniel,14540,Bourguebus,478,2004-10-22,5.0,4.8,...,1.0,T-shirt Blanc,L,170,60.0,0,Carton Tete de menagere,0,0,14
2,446,Mme,Chretien,Daniel,14540,Bourguebus,478,2004-10-22,5.0,4.8,...,1.0,Montre,Homme,30,150.0,0,Carton Tete de menagere,0,0,14
3,17860,M.,Verardo,Anthony,35400,Saint Malo,21239,2006-10-03,0.0,3.9,...,1.0,T-shirt Blanc,XL,180,60.0,0,Distingo 500 g,34,0,35
4,1330,Mme,Robert,Yvonne,61000,Alencon,1386,2004-11-23,4.0,6.5,...,1.0,Tete de menagere,Confidence,250,100.0,0,Carton Tete de menagere,0,0,61


In [32]:
df.dtypes

codcli         int32
genrecli      object
nomcli        object
prenomcli     object
cpcli         object
villecli      object
codcde         int32
datcde        object
timbrecli    float64
timbrecde    float64
Nbcolis      float64
cheqcli      float64
barchive       int64
bstock         int64
codobj         int32
qte            int32
Colis        float64
libobj        object
Tailleobj     object
Poidsobj       int64
points       float64
indispobj      int64
libcondit     object
prixcond       int64
puobj          int64
DEP           object
dtype: object

#### Doublons

In [33]:
masque_doublons = df.duplicated()
df_doublons = df[masque_doublons].copy()
df_doublons.loc[:, 'raison'] = 'Doublon'

In [34]:
# ajouter ces lignes au df_supprimé
df_supprimé = pd.concat([df_supprimé, df_doublons])

In [35]:
df = df.drop_duplicates()

#### Mise en forme

In [36]:
# changer l'ordre des colonnes
df=df[['codcde','datcde','codobj', 'libobj','qte', 'timbrecli',
       'timbrecde','codcli','nomcli','prenomcli','cpcli','villecli','DEP']]

In [37]:
# Trier par numéro de commande
df = df.sort_values('codcde').reset_index(drop=True)

In [38]:
df['timbrecde']=df['timbrecde'].astype(float)

In [39]:
# Enregistrement du csv
df.to_csv('DW.csv', index=False)

In [40]:
df_supprimé.to_csv('DW_lines_deleted.csv', index=False)